In [ ]:
# !KERAS_BACKEND=tensorflow python -c "from keras import backend"

from keras.models import load_model 

model= load_model('poverty_pred_3.h5')


In [ ]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir = '/home/aind2/Poverty_prediciton/figures/'

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'valid')

test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(256, 256),
        batch_size=20,
        class_mode='categorical')

test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('Test accuracy: %.2f%%' % test_acc)


In [ ]:
prediction = model.predict_generator(test_generator,8)

prediction[prediction>=0.5] = 1
prediction[prediction<0.5] = 0


In [ ]:

print (prediction)

In [ ]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob


# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), 3)
    return files, targets

# load train, test, and validation datasets
test_files, test_targets = load_dataset('/home/aind2/Poverty_prediciton/figures/valid')



# print (np.argmax(test_targets, axis=1))
# print (np.argmax(prediction, axis=1))

In [ ]:
from sklearn.metrics import  confusion_matrix
CM = confusion_matrix((np.argmax(test_targets, axis=1)),(np.argmax(prediction, axis=1)))
# print (CM )

In [ ]:
import matplotlib.pyplot as plt
import itertools

names=['middle ','poor','rich']


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
#     plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plot_confusion_matrix(CM, classes=names,
                      title='Confusion matrix, without normalization')

plt.show()